In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

tf.enable_eager_execution()
tf.__version__

# Softmax classification

* 임의의 Dataset 준비
* 3개의 클래스로 분류할 데이터 준비


In [ ]:
x_data = [[1., 2., 1., 1.],
          [2., 1., 3., 2.],
          [3., 1., 3., 4.],
          [4., 1., 5., 5.],
          [1., 7., 5., 5.],
          [1., 2., 5., 6.],
          [1., 6., 6., 6.],
          [1., 7., 7., 7.]] # 8x4
y_data = [[0., 0., 1.],
          [0., 0., 1.],
          [0., 0., 1.],
          [0., 1., 0.],
          [0., 1., 0.],
          [0., 1., 0.],
          [1., 0., 0.],
          [1., 0., 0.]] # 8x3

x_test = [[1., 1., 1., 1.]]
y_test = [[0., 0., 1.]]


## 임의의 Data를 이용해서 3개의 클래스를 가지는 데이터셋 생성

In [ ]:
#dataset을 선언합니다.
dataset = tf.data.Dataset.from_tensor_slices((x_data, y_data))
dataset = dataset.repeat(1).batch(8)

nb_classes = 3 # class의 개수입니다.

print(tf.Variable(x_data))
print(tf.Variable(y_data))

In [ ]:
#Weight and bias setting
W = tf.Variable(tf.random_normal([4, nb_classes]), name='weight')
b = tf.Variable(tf.random_normal([nb_classes]), name='bias')
variables = [W, b]

print(W,b)

# 가설 설정

* 가설에서 예측한 값들을 이용해 예측값들을 확률로 표현한다.

## $$ y_k = \frac{exp(x_k)}{\sum_{i=1}^{n}exp(x_i)}  $$

In [ ]:
# tf.nn.softmax computes softmax activations
# softmax = exp(logits) / reduce_sum(exp(logits), dim)
def hypothesis_softmax(X):
    return tf.nn.softmax(tf.matmul(X, W) + b)

print(hypothesis_softmax(tf.Variable(x_data)))

## 가설을 검증할 Cost 함수를 정의합니다
$$
\begin{align}
cost(h(x),y) & = −log(h(x))  &  if  &  y=1 \\\\\
cost(h(x),y) & = -log(1−h(x))  &  if  &  y=0
\end{align}
$$

### 두 식을 한번에 쓰게되면,

$$
\begin{align}
cost(h(x),y) & = −y log(h(x))−(1−y)log(1−h(x))
\end{align}
$$

In [ ]:
def loss_fn(hypothesis, labels):
    cost = -tf.reduce_mean(labels * tf.log(hypothesis) + (1 - labels) * tf.log(1 - hypothesis))
    return cost

optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.01)

### 학습 진행

In [ ]:
epochs = 5000

for step in range(epochs):
  for features, labels in dataset:
    with tf.GradientTape() as tape:
      loss_value = loss_fn(hypothesis_softmax(features),labels)
      grads = tape.gradient(loss_value, [W,b])
      optimizer.apply_gradients(grads_and_vars=zip(grads,[W,b]))
      if step % 100 == 0:
            print("Iter: {}, Loss: {:.4f}".format(step, loss_fn(hypothesis_softmax(features),labels)))

## Sample 데이터를 넣고 테스트해봅시다.

In [ ]:
sample_data = [[2,1,3,2]] # answer_label [[0,0,1]]
sample_data = np.asarray(sample_data, dtype=np.float32)

a = hypothesis_softmax(sample_data)

print(a)
print(tf.argmax(a, 1)) #index: 2

## 데이터를 이용해서 예측

In [ ]:
b = hypothesis_softmax(x_data)
print(b)
print(tf.argmax(b, 1))
print(tf.argmax(y_data, 1)) # matches with y_data


In [ ]:
def accuracy_fn(hypothesis, labels):
    predicted = tf.cast(hypothesis > 0.5, dtype=tf.float32)
    accuracy = tf.reduce_mean(tf.cast(tf.equal(predicted, labels), dtype=tf.float32))
    return accuracy
  
test_acc = accuracy_fn(hypothesis_softmax(x_data),y_data)
print("Testset Accuracy: {:.4f}".format(test_acc))